# Environment Setting

In [ ]:
import boto3

import pandas as pd
import numpy as np
import os


import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("..")

from src.utils import S3Utils, athena_to_pandas
from src.pipelines.shapefile_processor import ItalianCensusAreas

### Process Census Features

In [6]:
%%time

# Input files

S3 = S3Utils('gimi-data', region = 'eu-south-1')
directory = 'in/italy/census-areas-features/2011'
paths = S3.bucket_content_keys(directory = directory)

CPU times: user 111 ms, sys: 38.6 ms, total: 149 ms
Wall time: 868 ms


In [15]:
temp.head()

,CODREG,REGIONE,CODPRO,PROVINCIA,CODCOM,COMUNE,PROCOM,SEZ2011,NSEZ,ACE,...,E22,E23,E24,E25,E26,E27,E28,E29,E30,E31
0,1,Piemonte,1,Torino,1,Agliè,1001,10010000001,1,0,...,63,26,14,8,1,625,52,193,59,9
1,1,Piemonte,1,Torino,1,Agliè,1001,10010000002,2,0,...,50,13,6,5,0,375,62,125,25,0
2,1,Piemonte,1,Torino,1,Agliè,1001,10010000006,6,0,...,1,1,1,0,0,12,1,3,1,0
3,1,Piemonte,1,Torino,1,Agliè,1001,10010000007,7,0,...,17,9,3,0,0,110,20,36,3,0
4,1,Piemonte,1,Torino,1,Agliè,1001,10010000008,8,0,...,2,0,0,0,0,25,4,18,1,0


In [21]:
%%time


for path in paths[1:]:
    
    print(path)
    
    s3bucket = 'gimi-data'
    input_path = path 
    output_path = 'out/italy/census-areas-features-2011/'
        
    # Download and unzip file
    fname = path.split('/')[-1]
    fname_folder = fname.split('.')[0]
    S3.download_file(path, fname)
    
    # Read file from local
    temp = pd.read_csv(fname, sep = ';', encoding = 'iso-8859-1')
    
    # Create ouptup parquet file
    reg = path.split("/")[-1].split("_")[0]
    fname_parquet = "region={}.parquet".format(reg)
    temp['region'] = reg
    temp.to_parquet(fname_parquet)

    # Save to S3
    S3 = S3Utils('gimi-data', region = 'eu-south-1')
    S3.upload_file(fname_parquet, os.path.join(output_path, fname_parquet))
    
    # Clean up folder
    os.remove(fname)
    os.remove(fname_parquet)

in/italy/census-areas-features/2011/R01_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R02_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R03_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R04_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R05_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R06_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R07_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R08_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R09_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R10_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R11_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R12_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R13_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R14_indicatori_2011_sezioni.csv
in/italy/census-areas-features/2011/R15_indicato